In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
df = pd.read_csv('/content/sample_data/train.csv')

In [27]:
new_df = df.sample(30000)

In [30]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
63266,63266,110134,110135,Why can't we all just make this world a better...,How can we make world a better place to live?,0
293875,293875,415672,415673,Should I prefer Nirali or Tech-Max for diploma...,I'm in first year mechanical engineering. What...,0
338725,338725,21279,65422,What is the QuickBooks technical support phone...,Which is the best QuickBooks Tech Support Numb...,1
134360,134360,4478,60581,Which laptop is best to buy within the range o...,Which is the best laptop to buy under 30k?,1
350922,350922,479709,479710,How do I access my college wifi IDs?,How do I connect to my college wifi?,0


In [31]:
new_df.shape

(30000, 6)

In [32]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 63266 to 38113
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            30000 non-null  int64 
 1   qid1          30000 non-null  int64 
 2   qid2          30000 non-null  int64 
 3   question1     30000 non-null  object
 4   question2     30000 non-null  object
 5   is_duplicate  30000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 1.6+ MB


In [34]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [35]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
63266,Why can't we all just make this world a better...,How can we make world a better place to live?
293875,Should I prefer Nirali or Tech-Max for diploma...,I'm in first year mechanical engineering. What...
338725,What is the QuickBooks technical support phone...,Which is the best QuickBooks Tech Support Numb...
134360,Which laptop is best to buy within the range o...,Which is the best laptop to buy under 30k?
350922,How do I access my college wifi IDs?,How do I connect to my college wifi?


In [36]:
from sklearn.feature_extraction.text import CountVectorizer
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [37]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [38]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [39]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
63266,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
293875,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
338725,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
134360,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
350922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=2)

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7288333333333333

In [43]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7166666666666667